In [5]:
from ctypes import *              # import the C compatible data types
import time
from sys import platform, path    # this is needed to check the OS type and get the PATH
from os import sep                 # OS specific file path separators
import matplotlib.pyplot as plt
import numpy
import datetime
import os
import sys


In [6]:
if sys.platform.startswith("win"):
    dwf = cdll.dwf
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
else:
    dwf = cdll.LoadLibrary("libdwf.so")

constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
path.append(constants_path)
import dwfconstants as constants

In [9]:
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value) # flags to show usb devices only
# filter_flags = c_int(filter_flags.value | constants.enumfilterDemo.value) # flags to include demo devices ; comment this line to exclude
# Enumerate devices
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))
print(f"Number of devices found: {device_count.value}")
# Retrieve device name (optional)
name_buf = create_string_buffer(64) # buffer to store device name
sn_buf = create_string_buffer(32) # buffer to store device serial nmber
opn_status = c_int() # to store device opening status
status_lookup = lambda code: {
    0: "Not opened",
    1: "Opened by current process",
    2: "Opened by another process"
}.get(code, "Unknown status")

for i in range(device_count.value):
    dwf.FDwfEnumDeviceName(c_int(i), name_buf)
    dwf.FDwfEnumSN(c_int(i), sn_buf)
    dwf.FDwfEnumDeviceIsOpened(c_int(i),byref(opn_status))
    print(f"Detected device {i}\nname: {name_buf.value.decode()}\nserial: {sn_buf.value.decode()}\n{status_lookup(opn_status.value)}\n")
hdwf = c_int()
idx=int(input())
dwf.FDwfEnumDeviceIsOpened(c_int(idx),byref(opn_status))
if(opn_status.value==0):
    dwf.FDwfDeviceOpen(c_int(idx), byref(hdwf))
    if hdwf.value == 0:
        raise RuntimeError("Failed to open device.")
    print(f"device opened successfully!")
else:
    print(status_lookup(opn_status.value))

Number of devices found: 1
Detected device 0
name: Analog Discovery 3
serial: SN:210415BC9A47
Not opened

device opened successfully!


In [8]:
dwf.FDwfAnalogInReset(hdwf)
dwf.FDwfDeviceCloseAll()
dwf.FDwfDeviceClose(hdwf)

0

In [ ]:
dwRead=c_uint32()
# the device will only be configured when FDwf###Configure is called
dwf.FDwfDeviceAutoConfigureSet(hdwf, c_int(1))

# enable output/mask on 8 LSB IO pins, from DIO 0 to 7
dwf.FDwfDigitalIOOutputEnableSet(hdwf, c_int(0x0004))
# set value on enabled IO pins

# fetch digital IO information from the device 
dwf.FDwfDigitalIOStatus(hdwf)
# read state of all pins, regardless of output enable
dwf.FDwfDigitalIOInputStatus(hdwf, byref(dwRead))

iNak = c_int()

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3)) # 100kHz
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0)) # SCL = DIO-0
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1)) # SDA = DIO-1
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))
if iNak.value == 0:
    print("I2C bus error. Check the pull-ups.")
    quit()
time.sleep(1)

rgTX = (c_ubyte*16)(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15)
rgRX = (c_ubyte*16)()

print("Separate Write and Read:")
#                                8bit address  
dwf.FDwfDigitalI2cWrite(hdwf, c_int(0x1D<<1), rgTX, c_int(16), byref(iNak)) # write 16 bytes
if iNak.value != 0:
    print("Write NAK "+str(iNak.value))
time.sleep(0.1)

dwf.FDwfDigitalI2cRead(hdwf, c_int(0x1D<<1), rgRX, c_int(16), byref(iNak)) # read 16 bytes
if iNak.value != 0:
    print("Read NAK "+str(iNak.value))
print(list(rgRX))
time.sleep(0.1)

print("Write and Read with reStart:")
dwf.FDwfDigitalI2cWriteRead(hdwf, c_int(0x1D<<1), rgTX, c_int(1), rgRX, c_int(16), byref(iNak)) # write 1 byte restart and read 16 bytes
if iNak.value != 0:
    print("Read NAK "+str(iNak.value))
print(list(rgRX))

In [20]:
dwf.FDwfAnalogIOEnableSet(hdwf, True)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_ubyte(2), c_ubyte(0), c_double(5.0))  # Channel 2, node 0 = V+, enable

# Set voltage to 5V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_ubyte(1), c_ubyte(0), c_double(1))  # Node 0 = V+

0